In [1]:
import datetime, timedelta
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from sklearn import metrics
from sklearn import preprocessing

## Part 2
If you made it up to here all by yourself, you can use your prepared dataset to train an Algorithm of your choice to forecast whether it will snow on the following date for each station in this dataset:

In [2]:
import datetime, timedelta
# +3 leap years, -1 - tomorrow 
str(datetime.datetime.today()- datetime.timedelta(days=11*365 + 3 - 1)).split(' ')[0]

'2010-05-25'

You are allowed to use any library you are comfortable with such as sklearn, tensorflow keras etc. 
If you did not manage to finish part one feel free to use the data provided in 'coding_challenge.csv' Note that this data does not represent a solution to Part 1. 

In [3]:
# Load dataframes from .csv-files (saved in Part 1)
weather_data_cleaned_all = pd.read_csv('weather_data_cleaned_all.csv', sep=',', index_col=0).sort_index().reset_index(drop=True)
train = pd.read_csv('train.csv', sep=',', index_col=0).sort_index().reset_index(drop=True)
_eval = pd.read_csv('_eval.csv', sep=',', index_col=0).sort_index().reset_index(drop=True)
test = pd.read_csv('test.csv', sep=',', index_col=0).sort_index().reset_index(drop=True)

In [4]:
weather_data_cleaned_all['date'] = pd.to_datetime(weather_data_cleaned_all.date)
train['date'] = pd.to_datetime(train.date)
_eval['date'] = pd.to_datetime(_eval.date)
test['date'] = pd.to_datetime(test.date)

test_new = test.append(_eval, ignore_index=True)

In [5]:
# +3 leap years, -1 - tomorrow 
wanted_date = str(datetime.datetime.today()- datetime.timedelta(days=11*365 + 3 - 1)).split(' ')[0]
print("Wanted date: " + wanted_date + ".")

Wanted date: 2010-05-25.


In [6]:
all_dates = weather_data_cleaned_all["date"].dt.date

min_date = str(all_dates.min())
max_date = str(all_dates.max())
print("The available dates are between " + min_date + " and " + max_date + ".")
print("So, the latest date is " + max_date + " and it is earlier as our wanted date " + wanted_date + ".")

The available dates are between 2005-01-01 and 2010-04-15.
So, the latest date is 2010-04-15 and it is earlier as our wanted date 2010-05-25.


Because of what we don't have any data for the day before the desired forecast day, we could try to generate this data. Unfortunately, only 6 years of data are available, which is not enough to generalize. But we can try, for example, to calculate the mean or median of values from known years for float values, and for boolean use the values that occur most frequently in earlier years.
But in the algorithms below I used the data from training and test sets that I saved in part 1.

Another possibility would be to use only 'station_number' and 'date' as features, but then we ignore all weather parameters, which cannot be a good solution. 

In [7]:
'''
I change the datatype of 'station_number' from int to category, 
because these data are discrete, not continuous.

Another possibility would be to divide the train and test sets into subsets, 
individually for each station; 
and then fit each model with data from each station separately. 
But in this case we will have too small data and cannot generalize weather-data combinations 
between the stations. 
'''
weather_data_cleaned_all['station_number'] = weather_data_cleaned_all['station_number'].astype('category')
train['station_number'] = train['station_number'].astype('category')
_eval['station_number'] = _eval['station_number'].astype('category')
test['station_number'] = test['station_number'].astype('category')
test_new['station_number'] = test_new['station_number'].astype('category')

In [8]:
def create_train_test_np(train, test):
    
    train['date'] = train['date'].dt.dayofyear
    test['date'] = test['date'].dt.dayofyear
    
    feature_list = ['station_number',
                    'mean_temp',
                    'mean_dew_point',
                    'mean_sealevel_pressure',
                    'mean_visibility',
                    'mean_wind_speed',
                    'max_sustained_wind_speed',
                    'max_temperature',
                    'total_precipitation',
                    'fog',
                    'rain',
                    'snow',
                    'hail',
                    'thunder',
                    'tornado',
                    'date',
                    'snow_tomorrow']
    
    feature_list_ind = len(feature_list) - 1
    
    train_np = train.to_numpy()
    
    X_train = train_np[:, :feature_list_ind]
    y_train = train_np[:, feature_list_ind]
    
    test_np = test.to_numpy()
    
    X_test = test_np[:, :feature_list_ind]
    y_test = test_np[:, feature_list_ind]
    
    return X_train, y_train, X_test, y_test

In [9]:
def logistic_regression(X_train, y_train, X_test, y_test):
    
    X_train = preprocessing.scale(X_train)
    X_test = preprocessing.scale(X_test)

    clf = LogisticRegression(max_iter=3000)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    score = metrics.accuracy_score(y_test, y_pred)
    
    print("Logistic regression score:", score)

In [10]:
def svm_(X_train, y_train, X_test, y_test):
    
    clf = SVC()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    score = metrics.accuracy_score(y_test, y_pred)
    
    print("SVM score:", score)

In [11]:
def random_forest(X_train, y_train, X_test, y_test):
    
    clf = RandomForestClassifier()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    score = metrics.accuracy_score(y_test, y_pred)
    
    print("Random Forest score:", score)

In [12]:
X_train, y_train, X_test, y_test = create_train_test_np(train, test_new)

In [13]:
print("With original data:\n")
logistic_regression(X_train, y_train, X_test, y_test)
svm_(X_train, y_train, X_test, y_test)
random_forest(X_train, y_train, X_test, y_test)

With original data:

Logistic regression score: 0.8738738738738738
SVM score: 0.8706442291347952
Random Forest score: 0.8791432942376338


All these algorithms have accuracy score around 87%. But are they really so good?..

In [14]:
weather_data_cleaned_all['snow_tomorrow'].value_counts()

0    14631
1     2175
Name: snow_tomorrow, dtype: int64

The problem is the imbalance: we have much more days without snowfall as with snowfall. This means that even if these algorithms are always predicting "no snow", these predictions will be mostly correct.

To eliminate this imbalance, we can either delete random data with 'snow_tomorrow' == False, or copy random data with 'snow_tomorrow' == True several times, so that in the end there is the same amount of data with and without snow. Deleting will greatly reduce the number of data, so I'm trying to use the copy option.

In [15]:
def create_balanced_data(X_train, y_train, X_test, y_test):    
   
    y_train_1_idx = np.where(y_train == 1)[0] # Indices of data with 'snow_tomorrow' == True
    m_train = random.choices(y_train_1_idx, k=len(y_train) - len(y_train_1_idx))   
    
    X_train = np.append(X_train, X_train[m_train], axis=0)
    y_train = np.append(y_train, y_train[m_train], axis=0)
    
    y_test_1_idx = np.where(y_test == 1)[0] # Indices of data with 'snow_tomorrow' == True
    m_test = random.choices(y_test_1_idx, k=len(y_test) - len(y_test_1_idx))
    
    X_test = np.append(X_test, X_test[m_test], axis=0)
    y_test = np.append(y_test, y_test[m_test], axis=0)
    
    return X_train, y_train, X_test, y_test

In [16]:
X_train_balanced, y_train_balanced, X_test_balanced, y_test_balanced = create_balanced_data(X_train, y_train, X_test, y_test)

In [17]:
print("With balanced data:\n")
logistic_regression(X_train_balanced, y_train_balanced, X_test_balanced, y_test_balanced)
svm_(X_train_balanced, y_train_balanced, X_test_balanced, y_test_balanced)
random_forest(X_train_balanced, y_train_balanced, X_test_balanced, y_test_balanced)

With balanced data:

Logistic regression score: 0.6654248069059518
SVM score: 0.5345751930940481
Random Forest score: 0.5769195820081781


As we can see, the accuracies have become significantly worse, but at least they are over 50%.